# Numerical Methods Project 1 by Sercan Hüsnügil

We start by adding/checking the necessary packages.

In [1]:
] add CSV; 

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [2]:
] add DataFrames; 

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [3]:
] add Optim; 

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


Using the packages downloaded above.

In [4]:
using CSV # to extract data from .csv file
using DataFrames # to form/manipulate data frames
using Optim # to find bestfits for the data
using Statistics # to calculate mean, standard deviation etc.
using WGLMakie # for plots

## Question 1

1. A higher-order fit.  We did a linear fit (y_pred = b + m x).  Try extending the code to do a quadratic fit: y_pred = b + m x + q x^2. Does the fit improve?  Repeat the jack-knife analysis (and plot up the answers produced by each run of the jack-knife), and investigate the scatter in the "q" values.  Can you conclude anything about whether you need the "q" term for a good fit?

### a. Stating the problem

In [5]:
# Creating the data frame object we want to work on.
alldata = CSV.read("data.csv", DataFrame); # read data from .csv file
data = alldata[5:size(alldata,1), :]; # skip the first 4 data points as we did in class (so we have a nicer fit)

For a quadratic fit we need 3 parameters: $b + m x + q x^2$
- $b$ is the constant term
- $m$ is the coefficient of the linear term (cf. slope for the linear fit)
- $q$ is the coefficient of the quadratic term 

Basically, we are trying to fit a parabola to our data set.  
We begin by illustrating the difference between a linear fit and a quadratic fit by plotting them on the same figure.

In [6]:
# Initial parameters for quadratic fit, determined "by eye"
b_eye = 1; # constant term
m_eye = 0.5; # coefficient of the linear term
q_eye = 1/100; # coefficient of the quadratic term

In [7]:
# Parameters we used for a linear fit in class, just to compare.
m_lin = 2;
b_lin = 50;

In [8]:
# Plot the quadratic fit by eye and linear fit on the same plot.
f = Figure()
Axis(f[1, 1], xlabel="x", ylabel="y", title="Quadratic and Linear fits 'by eye'")
xx = LinRange(50, 250, size(data,1)) # x-axis for lines! plots
errorbars!(data.x, data.y, data.sigma_y)
scatter!(data.x, data.y, markersize = 10, color = :red)
yy_lin = m_lin.*xx .+ b_lin # linear fit
lines!(xx, yy_lin, color=:green)
yy_eye = q_eye.*xx.^2 .+ m_eye.*xx .+ b_eye # quadratic fit by eye
lines!(xx, yy_eye, color=:orange)
f

As we can see from the figure above, our initial guess for the quadratic fit does seem to be doing better than the linear fit. Therefore, we need to optimize the quadratic fit and hope that it will capture the behavior of the data better than the simple linear fit.

### b. Optimization

To repeat the optimization scheme we implemented in class, we start by defining a likelyhood function. Following the lectures, we implement a logarithm of Gaussian likelihood:

$p= \frac{1}{\sqrt{2 \pi} \sigma} \exp(-\frac{(y - y_{\textrm{pred}})^2}{2 \sigma}) \rightarrow \log(p)= -\log{(\sqrt{2 \pi}  \sigma)} -\frac{(y - y_{\textrm{pred}})^2}{2 \sigma}$

In [9]:
function gaussian_log_likelihood(x, mean, sigma)
    """
        Gaussian likelihood function as we implemented in class.
    """
    return sum(-log.(sigma * sqrt(2*π)) - 0.5 * (x - mean).^2 ./ sigma.^2)

end;

To feed the optimization function, we need to construct an objective function which is to be minimized. As the objective function is minimized, we want to achieve a better fit to our data, $\textit{i.e.,}$ the objective function should measure the difference between the $y$ values of the data set and the $y$ values predicted by the quadratic fit. 
We also define an objective function for linear fit just as we did in class for comparison.

In [10]:
function objective_gauss(params, x, y, sigma)
    """
        Quadratic fit objective function which uses the gaussian_log_likelihood function defined above.
    """
    b = params[1]; m = params[2]; q = params[3]; # extract the fit parameters from inputs
    y_pred = q.*x.^2 .+ m.*x .+ b # construct the predicted values from the quadratic fit
    return -gaussian_log_likelihood(y .+ 0., y_pred, sigma .+ 0.) # notice the - sign, this is to be minimized

end;

In [11]:
function objective_ling(params, x, y, sigma)
    """
        Linear fit objective function which uses the gaussian_log_likelihood function defined above.
    """
    b = params[1]; m = params[2]; # extract the fit parameters from inputs
    y_pred = m.*x .+ b # construct the predicted values from the linear fit
    return -gaussian_log_likelihood(y .+ 0., y_pred, sigma .+ 0.) # notice the - sign, this is to be minimized

end;

Now, with the Gaussian objective function defined above we can run the optimization scheme using "Optim.minimizer" function. 
We use "fit by eye" values for $b$ ,$m$ and $q$ defined in part a., as our starting parameters. 

In [12]:
# Quadratic fit
starting_params = [b_eye , m_eye, q_eye] .+ 0. # define starting parameters
result = optimize(p -> objective_gauss(p, data.x, data.y, data.sigma_y), starting_params) # running optimization
@assert Optim.converged(result) # check that if the optimization converged
b_opt, m_opt, q_opt = Optim.minimizer(result) # extract optimized values

# Linear fit
starting_params_lin = [b_lin , m_lin] .+ 0. # define starting parameters
result_lin = optimize(p -> objective_ling(p, data.x, data.y, data.sigma_y), starting_params_lin) # running optimization
@assert Optim.converged(result) # check that if the optimization converged
b_opt_lin, m_opt_lin= Optim.minimizer(result_lin); # extract optimized values

Finally, we can plot the optimized quadratic and linear fits.

In [13]:
f = Figure()
Axis(f[1, 1], xlabel="x", ylabel="y", title="Quadratic and Linear fits with optimized parameters")
xx = LinRange(50, 250, 20)
errorbars!(data.x, data.y, data.sigma_y)
scatter!(data.x, data.y, markersize = 10, color = :red)
yy_opt = q_opt.*xx.^2 .+ m_opt.*xx .+ b_opt
yy_lin = m_opt_lin.*xx .+ b_opt_lin
lines!(xx, yy_opt, color=:green)
lines!(xx, yy_lin, color=:orange)
f

It is hard to tell how much the quadratic term improves the fit just by inspecting the plot above by eye. We need a metric to quantify how good a fit is. We return to this problem in Stretch 1 below.

### c. Jack-knife method

In our data set, there can be some data points which does not follow the physical trend we wish to discover. Such data points might be resulting from experimental errors. Existance of these erronous measurements can mess up the quadratic fit we are implementing.

To circumvent this problem we utilize the jack-knife method. With the jack-knife method, we will leave out one data point and apply a quadratic fit to the remanining data. We repeat this procedure by leaving out one data point at a time, iterating over all the points in the data set.

In [35]:
# Jack-knife method.

n = length(data.x) # length of the data set

B_jack = zeros(n); M_jack = zeros(n); Q_jack = zeros(n); # creating vectors to store fit values

params = starting_params # use "fit by eye" parameters defined earlier

for i in 1:n  # iterating over every point in the data set 
    
    # leave out that point and add remaining to  _sub vectors
    x_sub = append!(data.x[1:(i-1)], data.x[(i+1):n])
    y_sub = append!(data.y[1:(i-1)], data.y[(i+1):n])
    s_sub = append!(data.sigma_y[1:(i-1)], data.sigma_y[(i+1):n])
    
    # optimize
    result = optimize(p -> objective_gauss(p, x_sub, y_sub, s_sub), params);
    @assert(Optim.converged(result)) # check that if the optimization converged
    b,m,q = Optim.minimizer(result)  # extract the optimized parameters
    
    B_jack[i] = b; M_jack[i] = m; Q_jack[i] = q; # add optimized parameters to the respective vectors
end

We plot the quadratic fits for every iteration of jack-knife. Therefore, we plot $n=20$ fits, each of which calculated over the remaining 19 data points.

In [37]:
# Plot quadratic fits for each iteration of jack-knife
f = Figure()

Axis(f[1, 1], xlabel="x", ylabel="y", title="Quadratic fits after jack-knife")
ndata = size(data,1)

errorbars!(data.x, data.y, data.sigma_y)
scatter!(data.x, data.y, markersize = 10, color = :red)

for i in 1:ndata
    yy_jack = xx.^2 .* Q_jack[i] .+ xx .* M_jack[i] .+ B_jack[i]
    lines!(xx, yy_jack, color=:green)
end

f

We also plot the values of the quadratic term coefficient $q$ versus the linear term coefficient $m$ for each jack-knife iteration. 

In [38]:
f = Figure()

Axis(f[1, 1], xlabel=L"m", ylabel=L"q", title="q vs m values in jack-knife iterations")
scatter!(M_jack, Q_jack, markersize = 10, color = :green)

f

From the scatter plot above, we see that the coefficent of the quadratic term $q$ varies in between the values $0.0015-0.0040$. However, as Dustin helped me to figure out, this interval is not scattered around 0. Therefore, we can infer that even if the coefficient is small, quadratic term is needed for a better fit.

### d. Stretch 1

Stretch 1. We were using (negative of) the log-likelihood for our objective function to optimize.  Part of that log-likelihood term is called the "chi-squared" term (or "sum of chi-squared"), and a change in chi-squared ("delta-chisq") is often used as a quick way of talking about how much a fit is improved by adding a parameter.  Read up about chi-squared and describe how much adding a quadratic term improved the fit, in terms of delta-chi-squared.  Finally, look up the Akaike Information Criterion, which is a numerical implementation of Occam's Razor that is supposed to tell you whether you are justified in adding the quadratic parameter to your model.  Does the AIC improve when you go from the linear to the quadratic model?  Optionally, look up the Bayesian Information Criterion (just a different formula for doing the same job); what does it say?

In our Gaussian log-likelihood function we have the term:

$\frac{(y - y_{\textrm{pred}})^2}{2 \sigma}$

The summation of which is defined as the $\chi^2$ value up to a factor [1]:

$ \chi ^ 2 \equiv \sum_i \frac{(y_i - y_\textrm{pred})^2}{\sigma_i^2} $

Let us calculate this value for our linear and quadratic fits and compare. (Note that we are using the sum of $\chi^2$ values. One can also implement different variations of it via normalizing etc.)

In [39]:
# sum of chi^2 for optimized quadratic fit
yy_opt = q_opt.*data.x.^2 .+ m_opt.*data.x .+ b_opt
chi_q = sum((data.y .- yy_opt).^2 ./ data.sigma_y.^2)

17.403171947980457

In [40]:
# sum of chi^2 for optimized linear fit
yy_lin = m_opt_lin.*data.x .+ b_opt_lin
chi_l = sum((data.y .- yy_lin).^2 ./ data.sigma_y.^2)

18.68076991240006

In [41]:
# calculate delta-chi-squared
delta_chi = (chi_q - chi_l)

-1.277597964419602

Comparing the values of $\chi^2$ (sum of $\chi^2$) we can say that the fit has improved when we include the quadratic term since the sum decreases. This means that the terms predicted by the quadratic fit are closer to the experimental data points compared to the prediction of the linear fit.

Finally, we can check Akaike Information Criterion (AIC) which quantifies how much a fit is improved by an addition of a parameter. AIC is calculated as [2]:
 
 $\textrm{AIC} = 2k-2\log(L_{\textrm{max}}) $
 
 where
 
 -$k$ is the number of parameters
 
 -$ L_{\textrm{max}}$ is the maximum of likelihood function.
 
 A "better fit" is quantified by a lower value of AIC.

In [42]:
# Calculate AIC for quadratic fit
L_q = maximum(-log.(data.sigma_y * sqrt(2*π)) - 0.5 * (data.y .- yy_opt).^2 ./ data.sigma_y.^2)
q_AIC = 2*3 - 2*(L_q)

13.254157864981625

In [43]:
# Calculate AIC for linear fit
L_l = maximum(-log.(data.sigma_y * sqrt(2*π)) - 0.5 * (data.y .- yy_lin).^2 ./ data.sigma_y.^2)
q_AIC = 2*2 - 2*(L_l)

11.616932482882584

To be able to say that adding the quadratic term is justifiable, we need to show that AIC decreases. However, we see that AIC for the linear fit is lower than the value for the quadratic fit. Therefore, even if the $\Delta_{\chi^2}$ value is in favor of the addition of the quadratic fit, AIC suggests that the improvement of the likelihood function does not compansate increasing the number of parameters in this case.

### References:

-[1]: CHI-SQUARE: TESTING FOR GOODNESS OF FIT, UCSC Physics. http://physics.ucsc.edu/~drip/133/ch4.pdf

-[2]: Akaike information criterion, Wikipedia. https://en.wikipedia.org/wiki/Akaike_information_criterion

## Question 2

2. Investigate the m,b plane.  In class we plotted up our solutions in the m,b coordinate plane.  Here, I'd like you to investigate a bit more.  Try making a contour plots of the (Gaussian) objective function (in terms of m,b), but giving it only a single data point.  (I think you should find that there is a diagonal locus of m,b values that have a maximum log-likelihood (so minimum objective-function value), because those lines go right through that single data point.  Now try the same thing with a different data point.  (Plot both contours on the same plot.  Then also try plotting the contour for the two data points combined.)  Now try all data points.  
Next, try plotting the contours for our outlier-rejecting version of the objective function.  Try expanding the range of the m and b parameters and see if you can find the "punk" minimum we found in class.  Are there other minima you can see also?

Here, we return to our discussion on the linear fit. This time, we want to explore $b-m$ space by contour plots.

In [44]:
# Modifying the code provided in PSI portal:
# Range of m,b values to plot
bvals = LinRange(0., 100., 100)
mvals = LinRange(2.0, 2.5, 100);
# Compute the objective function for each point in a grid for a single point.
og1 = [objective_ling([b,m], data.x[1], data.y[1], data.sigma_y[1])
      for b in bvals, m in mvals]
# Compute the objective function for each point in a grid for another point.
og2 = [objective_ling([b,m], data.x[10], data.y[10], data.sigma_y[10])
      for b in bvals, m in mvals]
# Compute the objective function for each point in a grid for the two points combined.
og3 = [objective_ling([b,m], [data.x[1], data.x[10]], [data.y[1], data.y[10]], [data.sigma_y[1], data.sigma_y[10]])
      for b in bvals, m in mvals]
f = Figure()
ax = Axis(f[1, 1], xlabel=L"b", ylabel=L"m", title="m vs b contour plots")
contour!(bvals, mvals, og1, levels=20, color="red", label = "data point 1 (red)")
contour!(bvals, mvals, og2, levels=20, color="blue", label = "data point 10 (blue)")
contour!(bvals, mvals, og3, levels=20, color="green", label = "data points combined (green)")
f[1,2] = Legend(f,ax, "Contours")
f


Now, we plot contours including all the data points. Here, I use "contourf" function to plot filled contours. Many thanks to Shahnewaz for tbe suggestion!

In [46]:
# Range of m,b values to plot
bvals = LinRange(0., 100., 100)
mvals = LinRange(2.0, 2.5, 100);
# Compute the objective function for each point in a grid
og_all = [objective_ling([b,m], data.x, data.y, data.sigma_y)
      for b in bvals, m in mvals]
f = Figure()
Axis(f[1, 1], xlabel=L"b", ylabel=L"m", title="m vs b contour plot (all points)")
contourf!(bvals, mvals, og_all, levels=20)
f

As we can see from the plots, there is a diagonal region which has the darkest color, $\textit{i.e.}$ maximum log likelihood. This is expected since it represents a single data point with the respective $b$ and $m$ values.

We repeat the contour plots for the outliers-rejecting objective function we implemented in the class.

In [47]:
function objective_outliers(parameters, x, y, sigma)
    """
        Objective outliers function using a linear fit. 
    """
    b = parameters[1]
    m = parameters[2]

    frac_bad = 0.01
    like_bad = frac_bad * (1. / 600.)

    y_pred = b .+ m .* x
    like_good = (1. - frac_bad) * 1 ./(sqrt(2*π) .* sigma) .* exp.(-0.5 * (y .- y_pred).^2 ./ sigma.^2)
    like = like_bad .+ like_good
    loglike = log.(like)

    return -sum(loglike)
end
     

objective_outliers (generic function with 1 method)

In [48]:
# Range of m,b values to plot
bvals = LinRange(-300., 300., 100)
mvals = LinRange(0., 6., 100);
# Compute the objective function for each point in a grid
og_all = [objective_outliers([b,m], data.x, data.y, data.sigma_y)
      for b in bvals, m in mvals]
f = Figure()
Axis(f[1, 1], xlabel=L"b", ylabel=L"m", title="m vs b contour plot (outliers)")
contourf!(bvals, mvals, og_all, levels=20)
f

With this method, we see the anticorrelation between $b-m$ more clearly. The minimum of the objectvie function value still occurs along a diagonal region. However, there is another diagonal branch on the left top part of the plot (slightly green on yellow background) which has smaller values with respect to the neighboring regions.

We investigate the correlation between $b$ and $m$ in the third question.

## Question 3

3. Remember that we found that our m,b estimates were anti-correlated. Can you rephrase or reparameterize the equation for a line so that the variables are less correlated?  First, you can try subtracting the average x value from all the x values, and find m',b' for that modified data set.  When you re-run the jack-knife test, does the covariance for the new m',b' values look better?  What happens if you try to convert your m',b' values back to m,b values for the original data set?

To reparametrize the line equation, we follow the suggested prescription.

We begin with: $y = mx + b$
- Substract average value of $x$ from all the $x$ values: $\tilde{x} = x - \bar{x}$ where $\bar{x} = \frac{1}{n} \sum_i^n x_i$. 
- Introduce $m'$ and $b'$ such that: $y = m' \tilde{x} + b'$

First, we apply jack-knife method for the line fit following the code we did in class. We also plot the scatter in $m-b$ values and print out the means and standard deviations.

In [49]:
# Jack-knife method without reparametrization.

n = length(data.x) # length of the data set

B_jack = zeros(n); M_jack = zeros(n); # creating vectors to store fit values

params = starting_params_lin # use "fit by eye" parameters defined earlier

for i in 1:n  # iterating over every point in the data set 
    
    # leave out that point and add remaining to  _sub vectors
    x_sub = append!(data.x[1:(i-1)], data.x[(i+1):n])
    y_sub = append!(data.y[1:(i-1)], data.y[(i+1):n])
    s_sub = append!(data.sigma_y[1:(i-1)], data.sigma_y[(i+1):n])
    
    # optimize
    result_lin = optimize(p -> objective_ling(p, x_sub, y_sub, s_sub), params)
    @assert(Optim.converged(result_lin)) # check that if the optimization converged
    b,m = Optim.minimizer(result_lin)  # extract the optimized parameters
    
    B_jack[i] = b; M_jack[i] = m; # add optimized parameters to the respective vectors
end

In [50]:
# Plot the linear fits for each iteration of jack-knife

f = Figure()

Axis(f[1, 1], xlabel="x", ylabel="y", title="Linear fits with jack-knife before reparametrization")
ndata = size(data,1)

errorbars!(data.x, data.y, data.sigma_y)
scatter!(data.x, data.y, markersize = 10, color = :red)

for i in 1:ndata
    yy_jack = xx .* M_jack[i] .+ B_jack[i]
    lines!(xx, yy_jack, color=:green)
end

f

In [51]:
# b-m scatter plot

f = Figure()

Axis(f[1, 1], xlabel=L"m", ylabel=L"b", title="b vs m values in jack-knife iterations")
scatter!(M_jack, B_jack, markersize = 10, color = :green)

f

From the scatter plot above, we see the diagonal (anti)correlation between $m$ and $b$ values as we explored in question 2 above.

In [52]:
# Some statistics on b-m values
print(" mean of b: ", mean(B_jack), "  standard deviation: ", std(B_jack)) 

print("\n mean of m: ", mean(M_jack), " standard deviation: ", std(M_jack)) 


 mean of b: 33.02991957534327  standard deviation: 7.806442030475723
 mean of m: 2.2450984101369267 standard deviation: 0.044395093198443876

We reapply the jack-knife method as earlier. This time, we redefine the $x$ values by substracting the average.

In [64]:
# Jack-knife method with reparametrization.

n = length(data.x) # length of the data set

B_jack_p = zeros(n); M_jack_p = zeros(n); # creating vectors to store fit values

params = starting_params_lin # use "fit by eye" parameters defined earlier

x_tilde = data.x .- sum(data.x) / size(data.x,1) # substract the average value of x

for i in 1:n  # iterating over every point in the data set 
    
    # leave out that point and add remaining to  _sub vectors
    x_sub = append!(x_tilde[1:(i-1)], x_tilde[(i+1):n])
    y_sub = append!(data.y[1:(i-1)], data.y[(i+1):n])
    s_sub = append!(data.sigma_y[1:(i-1)], data.sigma_y[(i+1):n])
    
    # optimize
    result_lin = optimize(p -> objective_ling(p, x_sub, y_sub, s_sub), params)
    @assert(Optim.converged(result_lin)) # check that if the optimization converged
    b_prime,m_prime = Optim.minimizer(result_lin)  # extract the optimized parameters
    
    B_jack_p[i] = b_prime; M_jack_p[i] = m_prime; # add optimized parameters to the respective vectors
end

In [65]:
# Plot the reparametrized linear fits for each iteration of jack-knife

f = Figure()

Axis(f[1, 1], xlabel="x", ylabel="y", title="Reparametrized linear fits with jack-knife")
ndata = size(data,1)

errorbars!(x_tilde, data.y, data.sigma_y)
scatter!(x_tilde, data.y, markersize = 10, color = :red)

xx_tilde = xx .- mean(xx); #LinRange(50, 250, size(data,1)) # x-axis for lines! plots

for i in 1:ndata
    yy_jack_p = xx_tilde .* M_jack_p[i] .+ B_jack_p[i]
    lines!(xx_tilde, yy_jack_p, color=:green)
end

f

In [66]:
# b-m scatter plot after reparametrization

f = Figure()

Axis(f[1, 1], xlabel=L"m'", ylabel=L"b'", title="b' vs m' values in jack-knife iterations")
scatter!(M_jack_p, B_jack_p, markersize = 10, color = :green)

f

Here, we see that the diagonal relation between $b$ and $m$ values are now destroyed. 

In [67]:
# Some statistics on b-m values
print(" mean of b': ", mean(B_jack_p), "  standard deviation: ", std(B_jack_p)) 

print("\n mean of m': ", mean(M_jack_p), " standard deviation: ", std(M_jack_p)) 
     

 mean of b': 409.6452414820975  standard deviation: 1.7185657071354807
 mean of m': 2.2450992102120932 standard deviation: 0.04439649007820377

Comparing the standard deviations of the parameters before and after reparametrization, one can notice that the distrutbion of $m$ and $m'$ values are similar to each other. On the other hand, the standard deviation of $b'$ values are lower than of $b$ values. Therefore, reparametrizing by method prescribed in the problem yielded a decrease in the variance of the constant term in the linar fit.

## Acknowledgements

Many thanks to Dustin, Rokas, Shahnewaz, Hassan, and Ali for helpful discussions. See the references at the end of Stretch 1 for the websites cited in that section. 